In [15]:
import sys
import socket
import time

import numpy as np
import matplotlib.pyplot as plt

## Connection to the detector

In [2]:
TCP_IP = '10.194.101.12' # Identifica il detector
TCP_PORT = 8002 # identifica la porta del detector
TCP_PORT_KERNEL = 8000 # identifica il kernel del detector
# porta 8000 è la porta al kernel del detector, serva in caso si voglia dire di spegnersi alla macchina
BUFFER_SIZE = int(2e5)
sOSA = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sKernel = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sOSA.connect((TCP_IP,TCP_PORT))
sKernel.connect((TCP_IP,TCP_PORT_KERNEL))

## Definition of functions

Functions for comunication

In [3]:
def SMS(message, sock):
    message += '\n'
    sock.sendall(message.encode('utf-8'))
    data = sock.recv(BUFFER_SIZE)
    data = data.decode('utf-8')
    return data

def SEND(message, sock):
    message += '\n'
    sock.sendall(message.encode('utf-8'))

def BUFFER(sock):
    message='CUR:BUFF?\n'
    sock.sendall(message.encode('utf-8'))
    data = sock.recv(BUFFER_SIZE).decode('utf-8')
    return data

Functions for units conversion

In [ ]:
c = 299792458 # m/s
def THz_to_nm(frequency):
    c = 299792458 # m/s
    wavelenght = c / frequency * 1e3
    return wavelenght

def nm_to_THz (wavelenght):
    frequency = c / wavelenght / 1e3
    return frequency

def twos_complement(hexstr, bits):
    value = int(hexstr, 16)
    if value & (1 << (bits - 1)):
        value -= 1 << bits
    return value

## Comunication with the detector

Check if the comunication with the detector is enstablished

In [4]:
MESSAGE = '*REM; *IDN?' # operazioni con * sono richieste di sistema
# MESSAGE = '*REM; *ESR?\n'
msg = SMS(MESSAGE,sKernel)
print(msg)

JDSU,MTS 8000,11047,ISU,7.32



Check if the detector is ready to acquire

In [5]:
while (SMS('*REM; STAT:ACQ?',sOSA) != 'STOPPED\n'):
        time.sleep(1)

Set acquisition range

In [6]:
SEND('OSASetup:SEAcq CSManual',sOSA)

lambda_0 = 1542 # Central peak of the laser
delta = 20 

center = str(nm_to_THz(lambda_0))
span = str(c / lambda_0**2 * delta / 1e3)

SEND('OSASetup:MCAcq ' + center,sOSA)
SEND('OSASetup:MSPAcq ' + span,sOSA)

Start acquisition

In [7]:
SEND('KEY STARt',sOSA)

Check if the acquisition has finished

In [8]:
while (SMS('STAT:ACQ?',sOSA) == 'IN_PROGRESS\n'):
    time.sleep(1)
    print(SMS('STAT:ACQ?',sOSA))

IN_PROGRESS

IN_PROGRESS

STOPPED



Take all the variables of the acquisition and check the measurement's units

In [10]:
SMS('CURve:XUN?', sOSA)

'THz\n'

In [11]:
SMS('CURve:YUN?', sOSA)

'dBm\n'

In [9]:
XOffset = float(SMS('CURve:XOFFset?', sOSA))
XScale = float(SMS('CURve:XSC?', sOSA))
YOffset = float(SMS('CURve:YOFFset?', sOSA))
YScale = float(SMS('CURve:YSC?', sOSA))

In [ ]:
curveSize = SMS('CUR:SIZE?', sOSA)

Download the buffer

In [20]:
def BUFFER(sock): # se funziona come deve è da spostare su
    message='CUR:BUFF?\n'
    sock.sendall(message.encode('utf-8'))
    nBits = sock.recv(2).decode('utf-8')
    print(nBits)
    nBits = int(nBits[1])
    print(nBits)

    nPoints = sock.recv(nBits).decode('utf-8')
    print(nPoints)
    nPoints = int(nPoints)
    print(nPoints)
    
    points = np.array([],dtype=int)

    if (nPoints == curveSize):
        for i in range(nPoints + 5):
            buff = sock.recv(4).decode('utf-8')
            if (buff == '\n'):
                print("The end of the buffer was reached")
                if (i == nPoints):
                    print("SUCCESS! The number of points found in the buffer is compatible with the number of points expected")
                else:
                    print("WARNING! The number of points found in the buffer doesn't correspond with the expected number of points")
            else:
                points = np.append(points,twos_complement(buff, 16))
                print(points)
    else:
        print("ERROR! The number of points in the buffer doesn't corresponds with the number of points acquired!")

    sock.close()
    print("Connection with the OSA was closed")
    time.sleep(0.2)
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((TCP_IP,TCP_PORT))
    SMS('*REM; STAT:ACQ?',sock)

In [ ]:
points = BUFFER(sOSA)

## Conversion of points into the correct measurement's units

In [ ]:
points = [YOffset + YScale * p for p in points]
# points

In [ ]:
plt.plot(points)

# Prove per BUFFER

In [2]:
buffer = '#6008000217F217E2180CBFFCB00\n'

In [7]:
nBits = buffer[:2]
print(nBits)
nBits = int(nBits[1])
print(nBits)

#6
6


In [10]:
nPoints = buffer[2:2+nBits]
print(nPoints)
nPoints = int(nPoints)
print(nPoints)

008000
8000


In [ ]:
for i in range(5):
    

In [17]:
points = np.array([],dtype=int)
for i in range(5):
    start = 2+nBits + i*4
    stop = start + 4
    points = np.append(points,twos_complement(buffer[start:stop], 16))
    print(points)


[8575]
[8575 8574]
[8575 8574 8576]
[  8575   8574   8576 -13313]
[  8575   8574   8576 -13313 -13568]


In [ ]:
sOSA.close()
time.sleep(0.2)
sOSA = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sOSA.connect((TCP_IP,TCP_PORT))
SMS(MESSAGE,sKernel)

In [ ]:
SMS('*REM; STAT:ACQ?',sOSA)

In [ ]:
# SMS('OSAS:RESO?', sOSA)

In [ ]:
len(data)

In [ ]:
data

# Decode del buffer se si scarica tutto assieme

In [ ]:
dataDec = data
# .decode('utf-8')

In [ ]:
nBytes = int(dataDec[1])
print('Number of char:', nBytes)

nPoints = int(int(dataDec[2:9])/4)
print('Points in the buffer:', nPoints)
# check che questo valore sia lo stesso di prima

def twos_complement(hexstr, bits):
    value = int(hexstr, 16)
    if value & (1 << (bits - 1)):
        value -= 1 << bits
    return value

points = np.array([],dtype=int)
for i in range(nPoints):
    start = 2+nBytes + i*4
    stop = start + 4
    points = np.append(points,twos_complement(dataDec[start:stop], 16))



Crea funzione di controllo che il numero di punti non superi il buffer richiesto da scaricare !!!!!!!!!!!!!!!!!!!!!!!

The data is given in frequency but the resolution is in nm so convert in nm and divide by resolution when calculating the power. 